In [7]:
from transformers import AutoTokenizer, AutoModel
import torch
import json
import os
import numpy as np
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-base-en-v1.5')
model = AutoModel.from_pretrained('BAAI/bge-base-en-v1.5')
model.eval()

with open("/kaggle/input/company-revenue/parsed_all_V2.json", "r") as json_file:
    data = json.load(json_file)

with open("/kaggle/input/company-revenue/search_100k_no_none.jsonl", "r") as json_file:
    raw_data = json.load(json_file)

def encode_text(text):
    encoded_input = tokenizer([text], padding=True, truncation=True, return_tensors='pt')

    with torch.no_grad():
        model_output = model(**encoded_input)
        sentence_embeddings = torch.mean(model_output[0][:, 0:], axis=1)
    
    return sentence_embeddings[0].tolist()
    
    
def encode(sample, raw_sample):
    sample["long_description"] = encode_text(raw_sample["long_description"])
    sample["business_tags"] = encode_text("\n".join(raw_sample["business_tags"]))
    sample["main_business_category"] = encode_text(raw_sample["main_business_category"])
    sample["technologies"] = encode_text("\n".join(raw_sample["technologies"]))

    return sample
    
for sample in tqdm(data):
    for raw_sample in raw_data:
        if sample["soleadify_id"] == raw_sample["soleadify_id"]:
            sample = encode(sample, raw_sample)
            break
        else:
            continue

with open("parsed_all_V4.jsonl", "w") as json_file:
    json.dump(data, json_file, indent=4)


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

100%|██████████| 53256/53256 [45:11<00:00, 19.64it/s]


In [7]:
import json
import numpy as np
from tqdm import tqdm
import copy

categories = [
    "long_description",
    "business_tags",
    "main_business_category",
    "technologies"
]

only_values = {}

with open("/kaggle/input/company-revenue/parsed_all_V2.1.jsonl", "r") as json_file:
    data = json.load(json_file)

with open("/kaggle/input/company-revenue/parsed_all_V2.2.jsonl", "r") as json_file:
    raw_data = json.load(json_file)

for sample, raw_sample in tqdm(zip(data, raw_data)):
    if raw_sample["soleadify_id"] != sample["soleadify_id"]:
        print("ID issue")
        break
    only_values[raw_sample["soleadify_id"]] = raw_sample["estimated_revenue"]
    for categorie in categories:
        raw_sample[categorie] = sample[categorie]

with open("parsed_all_V4.jsonl", "w") as json_file:
    json.dump(raw_data, json_file, indent=4)
    
with open("only_values.jsonl", "w") as json_file:
    json.dump(only_values, json_file, indent=4)



53256it [00:00, 156791.46it/s]
